# 2025 DL Lab8: RL Assignment_Super Mario World

**Your Answer:**    
Hi I'm XXX, XXXXXXXXXX.

## Overview
This project implements a **Deep Reinforcement Learning** pipeline to train an autonomous agent for Super Mario World. Leveraging the **Proximal Policy Optimization (PPO)** algorithm, the system interacts with the **stable-retro** environment to master the YoshiIsland1 level. Key components include a custom Vision Backbone for extracting features from raw pixel data and a suite of Environment Wrappers that handle frame preprocessing, action discretization, and reward shaping to facilitate efficient learning.

Reward function implement  
should do something in the beginning (monster attack)  
Custom PPO implement  
pre train weight 差不多，主要是 reward function  
model weight capacity 1GB  
class name 不要動 (可以新增，但是原本有的不要動)

## Imports

In [1]:
import os
import numpy as np
import retro
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize

from eval import evaluate_policy, record_video
from custom_policy import VisionBackbonePolicy, CustomPPO

/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [2]:
# Game Settings
GAME = "SuperMarioWorld-Snes"
STATE = "YoshiIsland1"

# Training Settings
TOTAL_STEPS = 13_107_200
TRAIN_CHUNK =    327_680
N_ENVS = 16
LEARNING_RATE = 1e-4

# Evaluation & Recording Settings
EVAL_EPISODES = 3
EVAL_MAX_STEPS = 18000
RECORD_STEPS = 1200

# Directories
LOG_DIR = "./runs_smw"
VIDEO_DIR       = os.path.join(LOG_DIR, "videos")
CKPT_DIR        = os.path.join(LOG_DIR, "checkpoints")
TENSORBOARD_LOG = os.path.join(LOG_DIR, "tb")

os.makedirs(LOG_DIR,   exist_ok=True)
os.makedirs(CKPT_DIR,  exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

## Environment Functions

In [3]:
from wrappers import make_base_env
def _make_env_thunk(game: str, state: str):
    """Return a function that creates an environment (for multiprocessing)."""
    def _thunk():
        return make_base_env(game, state)
    return _thunk

def make_vec_env(game: str, state: str, n_envs: int, use_subproc: bool = True):
    """Create a vectorized environment (multiple envs running in parallel)."""
    env_fns = [_make_env_thunk(game, state) for _ in range(n_envs)]
    
    if use_subproc and n_envs > 1:
        vec_env = SubprocVecEnv(env_fns)
    else:
        vec_env = DummyVecEnv(env_fns)

    return vec_env

## Initialize Env & Model

In [4]:
# 1. Create Training Environment
train_env = make_vec_env(GAME, STATE, n_envs=N_ENVS)
# train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10., clip_reward=10.)
print(f"Environment created: {GAME} - {STATE} with {N_ENVS} parallel envs.")

checkpoint_path = "None" # 6225920 (19) 有破壞
checkpoint_path = "runs_smw/checkpoints/P_RWD_18.zip"

best_mean = -1e18
trained = 0
round_idx = 0

# 2. Initialize Model
if os.path.exists(checkpoint_path):
    # 讀取現有模型
    model = CustomPPO.load(
        checkpoint_path, 
        env=train_env,
        device="cuda:0" # 確保使用 GPU
    )
    trained = model.num_timesteps
    round_idx = int(trained / TRAIN_CHUNK)
    print(f"[Sucess] Loaded model from {checkpoint_path}")
    print(f"trained: {trained}, round_index: {round_idx}")
else:
    print(f"[Fail] Can't load {checkpoint_path}. Will use new model")
    model = CustomPPO(
        VisionBackbonePolicy,
        train_env,
        policy_kwargs   = dict(normalize_images=False),
        n_epochs        = 4,
        n_steps         = 512,
        batch_size      = 512,
        learning_rate   = LEARNING_RATE,
        verbose         = 1,
        gamma           = 0.99,
        kl_coef         = 1,
        clip_range      = 0.125,
        tensorboard_log = TENSORBOARD_LOG,
    )

Environment created: SuperMarioWorld-Snes - YoshiIsland1 with 16 parallel envs.
[Sucess] Loaded model from runs_smw/checkpoints/P_RWD_18.zip
trained: 6225920, round_index: 19


In [5]:
# model.save("policy")

In [6]:
# import os
# import glob
# from custom_policy import CustomPPO
# from eval import record_video  # 確保 eval.py 在同一目錄下

# # ================= 設定區 =================
# # 遊戲設定 (請確保跟訓練時一致)
# # target_numbers = [3932160, 6225920, 12451840] 

# # 方法 B: 自動搜尋資料夾下所有 PIPE_{number}.zip (如果你想全部測的話，把下面解註解)
# files = glob.glob(os.path.join(CKPT_DIR, "SF84G_*.zip"))
# target_numbers = list(range(35, 40))

# # ================= 執行迴圈 =================
# print(f"準備測試以下 Checkpoints: {target_numbers}")

# for num in target_numbers:
#     model_path = os.path.join(CKPT_DIR, f"PIPE_{num}.zip")
    
#     # 檢查檔案是否存在
#     if not os.path.exists(model_path):
#         print(f"⚠️ 找不到檔案: {model_path}，跳過。")
#         continue
    
#     print(f"\n[{num}] 正在載入模型: {model_path} ...")
    
#     try:
#         # 1. 載入模型 (不需要 env 參數也能載入權重)
#         # 如果你有改過 CustomPPO 的參數，load 會自動讀取 zip 裡的設定
#         model = CustomPPO.load(model_path, device="auto") # device="auto" 會自動用 GPU
        
#         # 2. 錄製影片
#         prefix_name = f"test_{num}"
#         print(f"[{num}] 正在錄影 (長度 {RECORD_STEPS} steps)...")
        
#         record_video(
#             model=model,
#             game=GAME,
#             state=STATE,
#             out_dir=VIDEO_DIR,
#             video_len=RECORD_STEPS,
#             prefix=prefix_name
#         )
#         print(f"✅ 完成！影片已儲存為 {prefix_name}.mp4")
        
#     except Exception as e:
#         print(f"❌ 發生錯誤 (Model: {num}): {e}")

# print("\n所有測試結束。")

## Training Loop

In [ ]:
try:
    while trained < TOTAL_STEPS:
        round_idx += 1
        chunk = min(TRAIN_CHUNK, TOTAL_STEPS - trained)
        # chunk = 2000
        label = "P_RWD18"
        tagged_label = f"{label}_{int(trained/TRAIN_CHUNK)}"

        print(f"\n=== Round {round_idx} | Learn {chunk} steps (Total trained: {trained}) ===")
        
        # --- Train ---
        model.learn(total_timesteps=chunk, reset_num_timesteps=False, tb_log_name=label)
        trained += chunk

        # --- Save Checkpoint ---
        ckpt_path = os.path.join(CKPT_DIR, f"{tagged_label}.zip")
        model.save(ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

        # --- Evaluate ---
        mean_ret, best_ret = evaluate_policy(
            model,
            GAME,
            STATE,
            n_episodes=EVAL_EPISODES,
            max_steps=EVAL_MAX_STEPS,
        )
        print(f"[EVAL] Mean Return: {mean_ret:.3f}, Best Return: {best_ret:.3f}")

        # --- Save Best Model ---
        # if mean_ret > best_mean:
        #     best_mean = mean_ret
        #     best_path = os.path.join(LOG_DIR, "best_model.zip")
        #     model.save(best_path)
        #     print(f"New best record. Saved to {best_path}")

        # --- Record Video ---
        out_path = os.path.join(VIDEO_DIR, label)
        os.makedirs(out_path,  exist_ok=True)
        record_video(
            model,
            GAME,
            STATE,
            VIDEO_DIR,
            video_len=RECORD_STEPS,
            prefix=f"{label}/{tagged_label}_{mean_ret:.2f}",
        )

except KeyboardInterrupt:
    print("\nTraining interrupted manually.")

finally:
    train_env.close()
    print("Training finished. Environment closed.")
    
"""
tensorboard --logdir=./runs_smw/tb
"""


=== Round 20 | Learn 327680 steps (Total trained: 6225920) ===
Logging to ./runs_smw/tb/P_RWD18_0


/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

--------------------------------
| time/              |         |
|    fps             | 928     |
|    iterations      | 1       |
|    time_elapsed    | 8       |
|    total_timesteps | 6234112 |
--------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 823          |
|    iterations         | 2            |
|    time_elapsed       | 19           |
|    total_timesteps    | 6242304      |
| train/                |              |
|    approx_kl          | 0.0046068337 |
|    entropy_loss       | -2.33        |
|    explained_variance | 0.944        |
|    learning_rate      | 0.0001       |
|    loss               | 0.135        |
|    mean_step_reward   | 0.0104382355 |
|    n_updates          | 3044         |
|    policyGradLoss     | -0.002       |
|    value_loss         | 0.683        |
----------------------------------------
----------------------------------------
| time/                 |      

Saved checkpoint: ./runs_smw/checkpoints/P_RWD18_19.zip
[EVAL] Mean Return: -1.174, Best Return: -0.294
Saved video to ./runs_smw/videos/P_RWD18/P_RWD18_19_-1.17.mp4

=== Round 21 | Learn 327680 steps (Total trained: 6553600) ===
Logging to ./runs_smw/tb/P_RWD18_0
--------------------------------
| time/              |         |
|    fps             | 990     |
|    iterations      | 1       |
|    time_elapsed    | 8       |
|    total_timesteps | 6561792 |
--------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 815          |
|    iterations         | 2            |
|    time_elapsed       | 20           |
|    total_timesteps    | 6569984      |
| train/                |              |
|    approx_kl          | 0.0020536731 |
|    entropy_loss       | -2.34        |
|    explained_variance | 0.89         |
|    learning_rate      | 0.0001       |
|    loss               | 0.277        |
|    mean_st

Saved checkpoint: ./runs_smw/checkpoints/P_RWD18_20.zip
[EVAL] Mean Return: -0.544, Best Return: 1.056
Saved video to ./runs_smw/videos/P_RWD18/P_RWD18_20_-0.54.mp4

=== Round 22 | Learn 327680 steps (Total trained: 6881280) ===
Logging to ./runs_smw/tb/P_RWD18_0
--------------------------------
| time/              |         |
|    fps             | 1103    |
|    iterations      | 1       |
|    time_elapsed    | 7       |
|    total_timesteps | 6889472 |
--------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 838          |
|    iterations         | 2            |
|    time_elapsed       | 19           |
|    total_timesteps    | 6897664      |
| train/                |              |
|    approx_kl          | 0.0039384817 |
|    entropy_loss       | -2.28        |
|    explained_variance | 0.949        |
|    learning_rate      | 0.0001       |
|    loss               | 17.3         |
|    mean_ste

...

Saved checkpoint: ./runs_smw/checkpoints/P_RWD18_30.zip
[EVAL] Mean Return: -1.154, Best Return: -0.274
Saved video to ./runs_smw/videos/P_RWD18/P_RWD18_30_-1.15.mp4

=== Round 32 | Learn 327680 steps (Total trained: 10158080) ===
Logging to ./runs_smw/tb/P_RWD18_0
---------------------------------
| time/              |          |
|    fps             | 1283     |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 10166272 |
---------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 875          |
|    iterations         | 2            |
|    time_elapsed       | 18           |
|    total_timesteps    | 10174464     |
| train/                |              |
|    approx_kl          | 0.0022012386 |
|    entropy_loss       | -2.4         |
|    explained_variance | 0.922        |
|    learning_rate      | 0.0001       |
|    loss               | 0.213        |
|   

Saved checkpoint: ./runs_smw/checkpoints/P_RWD18_31.zip
[EVAL] Mean Return: -1.537, Best Return: -1.444
Saved video to ./runs_smw/videos/P_RWD18/P_RWD18_31_-1.54.mp4

=== Round 33 | Learn 327680 steps (Total trained: 10485760) ===
Logging to ./runs_smw/tb/P_RWD18_0
---------------------------------
| time/              |          |
|    fps             | 1027     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 10493952 |
---------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 830          |
|    iterations         | 2            |
|    time_elapsed       | 19           |
|    total_timesteps    | 10502144     |
| train/                |              |
|    approx_kl          | 0.0071380185 |
|    entropy_loss       | -2.23        |
|    explained_variance | 0.903        |
|    learning_rate      | 0.0001       |
|    loss               | 17.2         |
|   

Saved checkpoint: ./runs_smw/checkpoints/P_RWD18_32.zip
[EVAL] Mean Return: -1.255, Best Return: -0.521
Saved video to ./runs_smw/videos/P_RWD18/P_RWD18_32_-1.25.mp4

=== Round 34 | Learn 327680 steps (Total trained: 10813440) ===
Logging to ./runs_smw/tb/P_RWD18_0
---------------------------------
| time/              |          |
|    fps             | 993      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 10821632 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 888         |
|    iterations         | 2           |
|    time_elapsed       | 18          |
|    total_timesteps    | 10829824    |
| train/                |             |
|    approx_kl          | 0.008391628 |
|    entropy_loss       | -2.23       |
|    explained_variance | 0.932       |
|    learning_rate      | 0.0001      |
|    loss               | 11.3        |
|    mean_step_r

Saved checkpoint: ./runs_smw/checkpoints/P_RWD18_33.zip
[EVAL] Mean Return: -1.344, Best Return: -0.704
Saved video to ./runs_smw/videos/P_RWD18/P_RWD18_33_-1.34.mp4

=== Round 35 | Learn 327680 steps (Total trained: 11141120) ===
Logging to ./runs_smw/tb/P_RWD18_0
---------------------------------
| time/              |          |
|    fps             | 994      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 11149312 |
---------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 845          |
|    iterations         | 2            |
|    time_elapsed       | 19           |
|    total_timesteps    | 11157504     |
| train/                |              |
|    approx_kl          | 0.0035270182 |
|    entropy_loss       | -2.22        |
|    explained_variance | 0.913        |
|    learning_rate      | 0.0001       |
|    loss               | 7.71         |
|   

Saved checkpoint: ./runs_smw/checkpoints/P_RWD18_34.zip
[EVAL] Mean Return: -1.154, Best Return: -0.274
Saved video to ./runs_smw/videos/P_RWD18/P_RWD18_34_-1.15.mp4

=== Round 36 | Learn 327680 steps (Total trained: 11468800) ===
Logging to ./runs_smw/tb/P_RWD18_0


## Display Video

In [ ]:
from IPython.display import Video
import glob

list_of_files = glob.glob(os.path.join(VIDEO_DIR, '*.mp4')) 
if list_of_files:
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"Playing: {latest_file}")
    display(Video(latest_file, embed=True, width=600))
else:
    print("No videos found yet.")

In [ ]:
import cv2

cap = cv2.VideoCapture("runs_smw/videos/test_16.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Frame-by-Frame", frame)

    # 關鍵：這裡等待按鍵。按 'n' 鍵跳到下一幀，按 'q' 離開
    key = cv2.waitKey(0) 
    if key == ord('q'):
        break
    elif key == ord('n'):
        continue

cap.release()
cv2.destroyAllWindows()